# Oppgave 3: Databehandling

I denne oppgaven har jeg utviklet funksjoner som renser og formaterer de innsamlede albedo-dataene, med særlig fokus på håndtering av manglende verdier og uregelmessigheter ved hjelp av Pandas. Koden bruker list comprehensions, iteratorer og pandas SQL (sqldf).

---
- Gjorde om NetCDF4 til csv

For å redusere fil størrelsene og enklere bruk
400MB til 17MB

- Håndtering av manglede verdier og uregelmessigheter
Først identifiseres manglende eller ugyldige verdier i datasettet. For variabelen "AL-BB-DH" regnes både "NaN" og verdien "-1" som "missing value" (Kilde. produktmanualen). Rensefunksjonen bytter først ut alle "-1" med "NaN" og bruker deretter lineær interpolasjon ("interpolate()") for å fylle ut hull i dataserien.  
Dette gir **2 datasett** som kan potensiellt sammenlignes i videre analyse (dette gjøres ikke fordi lav feilmargin har null feil). 

---
På bakgrunn av hva vi lærte i oppgave 2 rundt datasettet har vi funnet ut:      
- Alpene er bra for å se på snøsmelting.
- At 21 juni er bra for å se på snøsmelting fra år til år, siden solen er på sitt sterkeste da.
- At AL-BB-DH passer vår oppgave best ved at vi ser på et tidspunkt med lite skydekke. 
- At AL-BB-DH-ERR gir også best data over de 21 årene, ved å sammenligne år til år. 
--- 

## Output fra kode 
 - Koden gir:
1. Sjekker at csv filene finnes(evt laster ned)
2. Sjekker om lavfeilmargin-fil finnes(evt laster ned)
3. Sjekker og renser datasettene med feil i seg

#Lavfeilmargin har 0 hull i seg i området vi utforsker. 

---

In [ ]:
import sys
sys.path.append("../src")

from albedo_effekt.databehandling import albedo_behandling
albedo_behandling()

✅ CSV-filer finnes fra før – hopper over ekstraksjon.
✅ Lavfeilmargin-fil finnes – sjekker kvalitet/interpolasjon.
🔍 Sjekker/utfører interpolasjon på lavfeilmargin-fil om nødvendig...
Albedo effekt 2004.csv: 0 ugyldige (<0/>1) ble satt til NaN. 5 manglende (-1/NaN) før, 0 etter interpolasjon. Rader: 5000 → 5000. Ny fil: Albedo effekt 2004_komplett.csv
Albedo effekt 2005.csv: 0 ugyldige (<0/>1) ble satt til NaN. 3 manglende (-1/NaN) før, 0 etter interpolasjon. Rader: 5000 → 5000. Ny fil: Albedo effekt 2005_komplett.csv
Albedo effekt 2006.csv: 0 ugyldige (<0/>1) ble satt til NaN. 1 manglende (-1/NaN) før, 0 etter interpolasjon. Rader: 5000 → 5000. Ny fil: Albedo effekt 2006_komplett.csv
Albedo effekt 2007.csv: 0 ugyldige (<0/>1) ble satt til NaN. 15 manglende (-1/NaN) før, 0 etter interpolasjon. Rader: 5000 → 5000. Ny fil: Albedo effekt 2007_komplett.csv
Albedo effekt 2008.csv: 0 ugyldige (<0/>1) ble satt til NaN. 2 manglende (-1/NaN) før, 0 etter interpolasjon. Rader: 5000 → 5000. Ny fi